In [1]:
# reload magic
%load_ext autoreload
%autoreload 2

In [ ]:

from mldec.datasets import toy_problem_data
import torch
from mldec.utils import evaluation

In [142]:
n = 8
batch_size = 10000
dataset_config = {
    'p': 0.15,
    'alpha': 0.33,
    'pcm': toy_problem_data.repetition_pcm(n),
    "sos_eos": (0, 0),
}
X, Y, weights = toy_problem_data.create_dataset_training(n, dataset_config)
weights_np = weights.numpy()

Xb, Yb, weightsb, downsampled_weights = toy_problem_data.sample_virtual_XY(weights_np, batch_size, n, dataset_config)
downsampled_weights_tensor = torch.tensor(downsampled_weights, dtype=torch.float32)

Xgood, Ygood, weightsgood = toy_problem_data.uniform_over_good_examples(n, dataset_config)

In [ ]:
from mldec.models.baselines import RepetitionCodeLookupTable, RepetitionCodeMinimumWeight



mld = RepetitionCodeLookupTable(n)
mld.train_on_histogram(Xgood, Ygood, weightsgood)

minimum_weight_decoder = RepetitionCodeMinimumWeight(n)
minimum_weight_decoder.make_decoder(X, Y)

In [153]:
print(Xb.shape)
print(X.shape, Y.shape, Ypred.shape)

torch.Size([120, 7])
torch.Size([256, 7]) torch.Size([256, 10]) torch.Size([120, 10])


In [162]:
lookup = RepetitionCodeLookupTable(n)
lookup.train_on_histogram(X, Y, downsampled_weights)


In [163]:


train_acc = evaluation.weighted_accuracy(lookup, X, Y, downsampled_weights_tensor) # training accuracy is evaluated on the same data from this epoch.
val_acc = evaluation.weighted_accuracy(lookup, X, Y, weights) # validation accuracy is evaluated on the full dataset
opt_val_acc = evaluation.weighted_accuracy(mld, X, Y, weights) # optimal validation accuracy is evaluated on the full dataset
minimum_weight_val_acc = evaluation.weighted_accuracy(minimum_weight_decoder, X, Y, weights) # optimal validation accuracy is evaluated on the full dataset
print("lookup train:", train_acc)
print("lookup test:", val_acc)
print("minimum weight:", minimum_weight_val_acc)
print("optimal:", opt_val_acc)


lookup train: 0.9994999170303345
lookup test: 0.9970260858535767
minimum weight: 0.997134804725647
optimal: 0.9988806843757629
